In [ ]:
%load_ext autoreload
%autoreload 2

: 

# Session 7: Complete Guide to SUMO and sumolib

## Table of Contents
1. [Introduction to SUMO and sumolib](#introduction)
2. [Setup and Installation](#setup)
3. [Importing sumolib](#importing)
4. [Creating and Loading Networks](#networks)
5. [Network Properties and Methods](#network-properties)
6. [Network Elements (Nodes, Edges, Lanes)](#elements)
7. [Practical Network Analysis Examples](#practical-examples)
8. [Route and Vehicle Generation](#routes)
9. [Parsing Route Files](#parsing-routes)
10. [Network Calculations](#calculations)
11. [Network Selection](#selection)
12. [Simulation Configuration](#config)
13. [TraCI Integration - Basics](#traci-basics)
14. [TraCI - Basic Workflow](#traci-workflow)
15. [TraCI - Simulation Control](#traci-control)
16. [TraCI - Adding Vehicles Dynamically](#traci-add-vehicles)
17. [TraCI - Subscriptions](#traci-subscriptions)
18. [TraCI - Context Subscriptions](#traci-context)
19. [TraCI - Subscription Filters](#traci-filters)
20. [TraCI - Vehicle Information](#traci-vehicle-info)
21. [TraCI - Vehicle Control](#traci-vehicle-control)
22. [Advanced Topics - Parallel Simulations](#parallel)
23. [Advanced Topics - Error Handling](#error-handling)
24. [Advanced Topics - Best Practices](#best-practices)
25. [Complete Example - Traffic Light Control](#complete-example)

---

## 1. Introduction to SUMO and sumolib <a name="introduction"></a>

**SUMO** (Simulation of Urban MObility) is an open source, highly portable, microscopic and continuous traffic simulation package designed to handle large networks.

**sumolib** is a set of Python modules for working with SUMO networks, simulation output, and other simulation artifacts. It provides:
- Network analysis and manipulation
- XML parsing utilities
- Coordinate transformations
- Route analysis
- Output processing
- Geometric calculations

**Key Features:**
- Load and analyze SUMO network files (`.net.xml`)
- Parse various SUMO XML files (routes, outputs, configurations)
- Perform coordinate transformations (geo, network, lane coordinates)
- Analyze traffic data and statistics
- Integrate with TraCI for runtime simulation control

**Documentation:**
- Official sumolib docs: https://sumo.dlr.de/docs/Tools/Sumolib.html
- PyDoc API reference: https://sumo.dlr.de/pydoc/sumolib.html
- TraCI interface: https://sumo.dlr.de/docs/TraCI/Interfacing_TraCI_from_Python.html

---

## 2. Setup and Installation <a name="setup"></a>

### Environment Setup

Before using sumolib, you need to ensure SUMO is installed and the `SUMO_HOME` environment variable is set correctly.

In [ ]:
import os
import sys

from sumolib import checkBinary
print(checkBinary("sumo"))
print(checkBinary("netconvert"))


: 

In [3]:
import os
import sys

# Check if SUMO_HOME is set
if 'SUMO_HOME' in os.environ:
    sumo_home = os.environ['SUMO_HOME']
    print(f"✓ SUMO_HOME is set to: {sumo_home}")
    
    # Add tools to Python path
    tools_path = os.path.join(sumo_home, 'tools')
    if tools_path not in sys.path:
        sys.path.append(tools_path)
    print(f"✓ Added tools directory to path: {tools_path}")
else:
    print("✗ SUMO_HOME environment variable is not set!")
    print("  Please set it to your SUMO installation directory")
    print("  Example (Linux/Mac): export SUMO_HOME=/usr/share/sumo")
    print("  Example (Windows): set SUMO_HOME=C:\\Program Files\\SUMO")

✗ SUMO_HOME environment variable is not set!
  Please set it to your SUMO installation directory
  Example (Linux/Mac): export SUMO_HOME=/usr/share/sumo
  Example (Windows): set SUMO_HOME=C:\Program Files\SUMO


---

## 3. Importing sumolib <a name="importing"></a>

The standard way to import sumolib in your scripts:

In [4]:
import sumolib
import sumolib.net  # For network operations
import sumolib.xml  # For XML parsing
import sumolib.geomhelper  # For geometric calculations
import sumolib.miscutils  # For utility functions
print("✓ Successfully imported sumolib submodules")

✓ Successfully imported sumolib submodules


---

## 4. Creating and Loading Networks <a name="networks"></a>

SUMO networks are stored in `.net.xml` files. These files contain the complete road network structure including nodes, edges, lanes, connections, and traffic lights.

### 4.1 Creating a Network File

You can create network files programmatically using custom generators:

In [5]:
from xml_generators import  SUMONetworkGenerator
import pandas as pd

network_nodes = pd.read_csv(r"~/Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/data/network/SiouxFallsXY.csv",
                            index_col=1)
# network_nodes
trafic_nodes = [str(i) for i in network_nodes.copy().index.tolist()]
# print(trafic_nodes)

network_nodes = network_nodes.apply(lambda x: {"x": float(x["x"]), "y": float(x["y"])}, axis=1).to_dict()
print("✓ Successfully loaded network nodes")
# network_nodes

network_edges = pd.read_csv(r"~/Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/data/network/SiouxFallsNetwork.csv")
# network_edges
network_edges = list(network_edges.apply(lambda x: {"from": int(x["from"]), "to": int(x["to"]),
                                               "length": float(x["length"])}, axis=1).values)
print("✓ Successfully loaded network edges")
# network_edges


xml_parsing = SUMONetworkGenerator(nodes=network_nodes, 
                                   edges=network_edges, 
                                   default_speed=80/3.6)
xml_parsing.add_traffic_lights_auto(green_time=30,
                                    yellow_time=5)
xml_parsing.save_with_netconvert(r"result.net.xml")

✓ Successfully loaded network nodes
✓ Successfully loaded network edges
Validated 24 nodes and 76 edges
Added traffic light to junction J2 with 4 phases
Added traffic light to junction J3 with 6 phases
Added traffic light to junction J1 with 4 phases
Added traffic light to junction J6 with 6 phases
Added traffic light to junction J4 with 6 phases
Added traffic light to junction J12 with 6 phases
Added traffic light to junction J5 with 6 phases
Added traffic light to junction J11 with 8 phases
Added traffic light to junction J9 with 6 phases
Added traffic light to junction J8 with 8 phases
Added traffic light to junction J18 with 6 phases
Added traffic light to junction J7 with 4 phases
Added traffic light to junction J16 with 8 phases
Added traffic light to junction J10 with 10 phases
Added traffic light to junction J15 with 8 phases
Added traffic light to junction J17 with 6 phases
Added traffic light to junction J14 with 6 phases
Added traffic light to junction J13 with 4 phases
Adde

RuntimeError: netconvert failed: Traceback (most recent call last):
  File "/Users/keivanjamali/miniconda3/envs/simulation-python/bin/netconvert", line 3, in <module>
    from sumo import netconvert
ImportError: cannot import name 'netconvert' from 'sumo' (/Users/keivanjamali/miniconda3/envs/simulation-python/lib/python3.11/site-packages/sumo/__init__.py)


---

## 5. Network Properties and Methods <a name="network-properties"></a>

### 5.1 Loading and Exploring the Network

Once loaded, a network object provides many useful methods to access and analyze the network structure.

In [21]:
net = sumolib.net.readNet('result.net.xml', withPrograms=True)

# Load with additional options
# net = sumolib.net.readNet('myNet.net.xml', 
#                           withPrograms=True,           # Import all traffic light programs
#                           withLatestPrograms=False,    # Import only last TL program
#                           withConnections=True,        # Import all connections (default)
#                           withFoes=True,              # Import right-of-way info (default)
#                           withInternal=False,         # Import internal edges/lanes
#                           withPedestrianConnections=False)  # Import sidewalk connections

print("# Edges")
edges = net.getEdges()
print(f"✓ Network has {len(edges)} edges")
print(edges[0])

print("############################################")
print("# Nodes")
nodes = net.getNodes()
print(f"✓ Network has {len(nodes)} nodes")
print(nodes[0])

print("############################################")
print("# Edge")
temp_edge = net.getEdge(id="E1_3")
print(temp_edge)

print("############################################")
print("# Node")
temp_node = net.getNode(id="J1")
print(temp_node)

print("############################################")
print("# Lane")
temp_lane = net.getLane(laneID="E1_3_0")
print(temp_lane.getID())
print(temp_lane.getLength())
print(temp_lane.getSpeed())

print("############################################")
tls = net.getTrafficLights()
print(f"✓ Network has {len(tls)} traffic lights")
i = 1
print(tls[i].getID())
print(tls[i].getEdges())
print(tls[i].getLinks())
print(tls[i].getPrograms()["0"])

# Edges
✓ Network has 76 edges
<edge id="E10_11" from="J10" to="J11"/>
############################################
# Nodes
✓ Network has 24 nodes
<junction id="J10"/>
############################################
# Edge
<edge id="E1_3" from="J1" to="J3"/>
############################################
# Node
<junction id="J1"/>
############################################
# Lane
E1_3_0
4000.0
22.22
############################################
✓ Network has 24 traffic lights
J10
{<edge id="E11_10" from="J11" to="J10"/>, <edge id="E9_10" from="J9" to="J10"/>, <edge id="E15_10" from="J15" to="J10"/>, <edge id="E16_10" from="J16" to="J10"/>, <edge id="E17_10" from="J17" to="J10"/>}
{20: [[<sumolib.net.lane.Lane object at 0x70a8ec52cfc0>, <sumolib.net.lane.Lane object at 0x70a8ec52cae0>, 20]], 21: [[<sumolib.net.lane.Lane object at 0x70a8ec52cfc0>, <sumolib.net.lane.Lane object at 0x70a8ec52cc80>, 21]], 22: [[<sumolib.net.lane.Lane object at 0x70a8ec52cfc0>, <sumolib.net.lane.Lane object at 0

---

## 6. Network Elements (Nodes, Edges, Lanes) <a name="elements"></a>

### 6.1 Working with Nodes (Junctions)

Nodes represent junctions or endpoints in the network.

In [22]:
# Get a specific node
node = net.getNode('J1')

# Node properties and methods:
node_id = node.getID() 
print(node_id)
coord = node.getCoord()                   # Get (x, y) coordinates
print(coord)
node_type = node.getType()                # Get node type (traffic_light, priority, etc.)
print(node_type)
incoming_edges = node.getIncoming()       # Get list of incoming edges
print(incoming_edges)
outgoing_edges = node.getOutgoing()       # Get list of outgoing edges
print(outgoing_edges)
connections = node.getConnections()       # Get all connections through this node
print(connections)


J1
(0.0, 460000.0)
traffic_light
[<edge id="E2_1" from="J2" to="J1"/>, <edge id="E3_1" from="J3" to="J1"/>]
[<edge id="E1_2" from="J1" to="J2"/>, <edge id="E1_3" from="J1" to="J3"/>]
[<sumolib.net.connection.Connection object at 0x70a8ec58a0a0>, <sumolib.net.connection.Connection object at 0x70a8ec58a360>]


### 6.2 Working with Edges

Edges represent road segments connecting nodes. Each edge can have multiple lanes.

In [23]:
# Get a specific edge
edge = net.getEdge('E1_3')

# Edge properties and methods:
edge_id = edge.getID()                    # Get edge ID
print(edge_id)
length = edge.getLength()                 # Get edge length in meters
print(length)
speed = edge.getSpeed()                   # Get maximum allowed speed (m/s)
print(speed)
priority = edge.getPriority()             # Get edge priority
print(priority)
from_node = edge.getFromNode()            # Get starting node
print(from_node)
to_node = edge.getToNode()                # Get ending node
print(to_node)
lanes = edge.getLanes()                   # Get list of lanes on this edge
print(lanes)
num_lanes = len(lanes)                    # Number of lanes
print(num_lanes)
shape = edge.getShape()                   # Get edge geometry as list of (x,y) points
print(shape)
lane_number = edge.getLaneNumber()        # Get number of lanes
print(lane_number)

# Edge connectivity
outgoing = edge.getOutgoing()             # Get edges that can be reached from this edge
incoming = edge.getIncoming()             # Get edges that lead to this edge
print(outgoing)
print(incoming)
# Check if edge allows specific vehicle class
allows_passenger = edge.allows('passenger')
allows_bicycle = edge.allows('bicycle')
print(allows_passenger)
print(allows_bicycle)

# Check if edge is selected (for analysis)
is_selected = edge.isSelected()
print(is_selected)

E1_3
4000.0
22.22
-1
<junction id="J1"/>
<junction id="J3"/>
1
[(-1.6, 459996.8), (-1.6, 390007.2)]
1
{<edge id="E3_12" from="J3" to="J12"/>: [<sumolib.net.connection.Connection object at 0x70a8ec57b800>], <edge id="E3_4" from="J3" to="J4"/>: [<sumolib.net.connection.Connection object at 0x70a8ec57b8b0>], <edge id="E3_1" from="J3" to="J1"/>: [<sumolib.net.connection.Connection object at 0x70a8ec57b960>]}
{<edge id="E2_1" from="J2" to="J1"/>: [<sumolib.net.connection.Connection object at 0x70a8ec58a0a0>]}
True
True
False


### 6.3 Working with Lanes

Lanes are the individual traffic lanes on an edge.

In [24]:
# Get a specific lane
lane = net.getLane('E1_3_0')
# or get from edge: lane = edge.getLane(0)

# Lane properties and methods:
lane_id = lane.getID()                    # Get lane ID
print(lane_id)
length = lane.getLength()                 # Get lane length in meters
print(length)
speed = lane.getSpeed()                   # Get maximum allowed speed (m/s)
print(speed)
width = lane.getWidth()                   # Get lane width in meters
print(width)
shape = lane.getShape()                   # Get lane geometry as list of (x,y) points
print(shape)
edge = lane.getEdge()                     # Get parent edge
print(edge)
index = lane.getIndex()                   # Get lane index on the edge (0-based)
print(index)


E1_3_0
4000.0
22.22
3.2
[(-1.6, 459996.8), (-1.6, 390007.2)]
<edge id="E1_3" from="J1" to="J3"/>
0


---

## 7. Practical Network Analysis Examples <a name="practical-examples"></a>

In [25]:
import sumolib

# Load the network
net = sumolib.net.readNet('result.net.xml')

# Example 1: Retrieve node coordinates
node = net.getNode('J1')
x, y = node.getCoord()
print(f"Node {node.getID()} is located at ({x}, {y})")

# Example 2: Get successor node of an edge
edge = net.getEdge('E1_3')
next_node = edge.getToNode()
next_node_id = next_node.getID()
print(f"Edge {edge.getID()} leads to node {next_node_id}")

# Example 3: Calculate total network length
total_length = 0.0
for edge in net.getEdges():
    total_length += edge.getLength()
print(f"Total network length: {total_length:.2f} meters")

# Example 4: Find edges with specific speed limit
high_speed_edges = []
speed_threshold = 30.0  # 30 m/s ≈ 108 km/h
for edge in net.getEdges():
    if edge.getSpeed() >= speed_threshold:
        high_speed_edges.append(edge.getID())
print(f"Found {len(high_speed_edges)} high-speed edges")

# Example 5: Analyze edge connectivity
edge = net.getEdge('E1_3')
print(f"Edge {edge.getID()} connects to:")
for successor_edge in edge.getOutgoing():
    print(f"  → {successor_edge.getID()}")

# Example 6: Count traffic light junctions
tls_count = 0
for node in sumolib.xml.parse("traffic_lights.tls.xml", ['tlLogic']):
    tls_count += 1

print(f"Network has {tls_count} traffic light junctions")

Node J1 is located at (0.0, 460000.0)
Edge E1_3 leads to node J3
Total network length: 314000.00 meters
Found 0 high-speed edges
Edge E1_3 connects to:
  → E3_12
  → E3_4
  → E3_1
Network has 24 traffic light junctions


---

## 8. Route and Vehicle Generation <a name="routes"></a>

Create routes and vehicle definitions for your simulation using route generators.

In [ ]:
from xml_generators import SUMORouteGenerator
import pandas as pd

# Create generator
route_gen = SUMORouteGenerator()

# Add vehicle type
route_gen.add_vehicle_type(type_id='car', 
                           accel=2.6, 
                           decel=4.5, 
                           length=5.0, 
                           max_speed=33.33, 
                           color="red")

# Read demand data from CSV
demand_df = pd.read_csv(r"~/Projects/Pure-Python/P5/08-Advanced_Traffic_TA_Class/data/network/demand.csv")
print(f"✓ Loaded {len(demand_df)} trips from demand.csv")
print(f"Demand data preview:\n{demand_df.head()}")

# Create trips from demand data
for idx, row in demand_df.iterrows():
    trip_id = f"trip_{int(row['player_id'])}"
    from_edge = f"E{int(row['source'])}_{int(row['source']) + 1}"  # Adjust edge naming as needed
    to_edge = f"E{int(row['destination'])}_{int(row['destination']) + 1}"  # Adjust edge naming as needed
    depart_time = int(row['arrival_time'])
    
    route_gen.add_trip(trip_id=trip_id, 
                      from_edge=from_edge, 
                      to_edge=to_edge, 
                      depart=depart_time, 
                      vtype='car', 
                      color="1,0,0")

print(f"✓ Created {len(demand_df)} trips")

# Save the route file
route_gen.save_xml('routes.rou.xml', begin=0, end=3600)
print("✓ Saved routes to routes.rou.xml")

Added vehicle type 'default'
Added vehicle type 'car'
Added trip 'trip_1' from E1_2 to E3_4 (dynamic routing)
Added STATIC route 'route_2' with 2 edges and flow 300 veh/h
Added 2 OD flows for DYNAMIC routing (shortest path)
  → Use 'duarouter' or '--device.rerouting.probability 1.0' for dynamic assignment
Routes saved to routes.rou.xml
  - 2 vehicle types
  - 1 static routes, 1 static flows
  - 1 dynamic trips, 2 dynamic flows (OD-based)

  ⚠️  For dynamic routing, run:
      duarouter -n network.net.xml -r routes.rou.xml -o routes_computed.rou.xml
      OR use: sumo --device.rerouting.probability 1.0 --device.rerouting.period 60


---

## 9. Parsing Route Files <a name="parsing-routes"></a>

Use sumolib to parse and analyze route files:

In [27]:
import sumolib

for route in sumolib.xml.parse_fast("routes.rou.xml", 'route', ['edges']):
    edge_ids = route.edges.split()
    print(f"Route has {len(edge_ids)} edges")

for vehicle in sumolib.xml.parse("routes.rou.xml", "vType"):
    print(f"Vehicle {vehicle.id} with the color of {vehicle.color}")

for trip in sumolib.xml.parse("routes.rou.xml", "trip"):
    print(f"trip: {trip.id}: departe={trip.depart}s, "
          f"Vehicle={trip.type}")

Route has 2 edges
Vehicle default with the color of yellow
Vehicle car with the color of red
trip: trip_1: departe=0s, Vehicle=car


---

## 10. Network Calculations <a name="calculations"></a>

A common task is to find the closest edge or lane to a given coordinate.

In [28]:
# import sumolib

# net = sumolib.net.readNet('result.net.xml')

# # Example: Find closest edge to a geo-coordinate
# lon, lat = 13.4050, 52.5200

# # Convert to network coordinates
# x, y = net.convertLonLat2XY(lon, lat)

# # Define search radius (in meters)
# radius = 1000_000.0 * 1000

# # Get neighboring edges
# edges = net.getNeighboringEdges(x, y, radius)

# # edges is a list of (edge, distance) tuples
# if len(edges) > 0:
#     # Sort by distance to find the closest
#     sorted_edges = sorted([(dist, edge) for edge, dist in edges], 
#                           key=lambda x: x[0])
    
#     closest_dist, closest_edge = sorted_edges[0]
#     print(f"Closest edge: {closest_edge.getID()}")
#     print(f"Distance: {closest_dist:.2f} meters")
#     print(f"Edge length: {closest_edge.getLength():.2f} meters")
#     print(f"Max speed: {closest_edge.getSpeed():.2f} m/s")
    
#     # Get all edges within radius
#     print(f"\\nFound {len(edges)} edges within {radius}m radius:")
#     for dist, edge in sorted_edges[:5]:  # Show top 5
#         print(f"  {edge.getID()}: {dist:.2f}m away")
# else:
#     print(f"No edges found within {radius}m radius")
#     print("Try increasing the search radius")


---

## 11. Network Selection <a name="selection"></a>

SUMO allows you to work with subsets of the network using selection files.

In [30]:
import sumolib

# Load network
net = sumolib.net.readNet('result.net.xml')

# Create a selection programmatically FIRST
# Example: Select all edges with speed > 2.0 m/s
high_speed_edges = []
for edge in net.getEdges():
    if edge.getSpeed() > 40.0:
        high_speed_edges.append(edge.getID())

# Write selection to file
with open('high_speed_selection.txt', 'w') as f:
    for edge_id in high_speed_edges:
        f.write(f"edge:{edge_id}\n")

print(f"Created selection with {len(high_speed_edges)} high-speed edges")

# NOW load the selection file
net.loadSelection('high_speed_selection.txt')

# Calculate cumulative length of selected edges
cumul_length = 0.0
selected_count = 0

for edge in net.getEdges():
    if edge.isSelected():
        cumul_length += edge.getLength()
        selected_count += 1

print(f"\nSelected {selected_count} edges")
print(f"Total length of selected edges: {cumul_length:.2f} meters")
if selected_count > 0:
    print(f"Average edge length: {cumul_length/selected_count:.2f} meters")

Created selection with 0 high-speed edges

Selected 0 edges
Total length of selected edges: 0.00 meters


---

## 12. Simulation Configuration <a name="config"></a>

Create SUMO configuration files programmatically:

In [31]:
from xml_generators import SUMOConfigGenerator

# Create config generator
config = SUMOConfigGenerator()

# Set input files
config.set_network('result.net.xml')
config.set_routes('routes.rou.xml')

# Set simulation time
config.set_time(begin=0, end=3600, step_length=1.0)

# Enable dynamic routing
config.enable_rerouting(probability=1.0, period=60)

# Add outputs
config.add_output('tripinfo', 'tripinfo.xml')
config.add_output('summary', 'summary.xml')

# Add detector
config.add_induction_loop('loop1', 'E1_2', pos=50)

# Save configuration
config.save('simulation.sumocfg')

Set network file: result.net.xml
Added route file: routes.rou.xml
Set simulation time: 0s to 3600s (step: 1.0s)
Enabled rerouting: probability=1.0, period=60s
Added output: tripinfo → tripinfo.xml
Added output: summary → summary.xml
Added induction loop 'loop1' on E1_2 at 50m
Added additional file: simulation.add.xml
Saved additional file: simulation.add.xml

Configuration saved to simulation.sumocfg
  Network: result.net.xml
  Routes: routes.rou.xml
  Time: 0s to 3600s (step: 1.0s)
  Rerouting: enabled (period=60s)
  Outputs: tripinfo, summary

  To run simulation:
      sumo -c simulation.sumocfg
      sumo-gui -c simulation.sumocfg  (with GUI)


In [16]:
raise

RuntimeError: No active exception to reraise

---

## 13. TraCI Integration - Basics <a name="traci-basics"></a>

**TraCI** (Traffic Control Interface) allows you to control and interact with a running SUMO simulation in real-time.

### TraCI vs sumolib:
- **sumolib**: Static analysis of network files and outputs (no simulation running)
- **TraCI**: Dynamic control of running simulation (real-time interaction)

### Installing and Importing TraCI

In [32]:
import os
import sys
if 'SUMO_HOME' in os.environ:
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
import traci

---

## 14. TraCI - Basic Workflow <a name="traci-workflow"></a>

In [33]:
import os
import sys
if 'SUMO_HOME' in os.environ:
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
import traci

# Step 1: Prepare the SUMO command
sumoBinary = "sumo-gui"  # Use "sumo-gui" for graphical interface
sumoCmd = [sumoBinary, "-c", "simulation.sumocfg"]

# Optional: Add additional SUMO options
sumoCmd += ["--step-length", "1"]  # Simulation step length
sumoCmd += ["--no-warnings", "true"]  # Suppress warnings

# Step 2: Start SUMO and connect TraCI
traci.start(sumoCmd)

# Step 3: Run simulation steps
step = 0
while step < 1000:
    # Advance simulation by one step
    traci.simulationStep()
    
    step += 1


 Retrying in 1 seconds


Fontconfig warning: "/usr/share/fontconfig/conf.avail/05-reset-dirs-sample.conf", line 6: unknown element "reset-dirs"


In [34]:
step = 0
while step < 1000:
    traci.simulationStep()
    
    step += 1



In [35]:
traci.close()

---

## 15. TraCI - Simulation Control <a name="traci-control"></a>

### 15.1 Retrieving Simulation State

In [17]:
import os
import sys
if 'SUMO_HOME' in os.environ:
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
import traci

# Step 1: Prepare the SUMO command
sumoBinary = "sumo-gui"  # Use "sumo-gui" for graphical interface
sumoCmd = [sumoBinary, "-c", "simulation.sumocfg"]

# Optional: Add additional SUMO options
sumoCmd += ["--step-length", "1"]  # Simulation step length
sumoCmd += ["--no-warnings", "true"]  # Suppress warnings

# Step 2: Start SUMO and connect TraCI
traci.start(sumoCmd)

# Step 3: Run simulation steps
step = 0
while step < 1000:
    # Advance simulation by one step
    traci.simulationStep()
    
    step += 1


 Retrying in 1 seconds


Fontconfig warning: "/usr/share/fontconfig/conf.avail/05-reset-dirs-sample.conf", line 6: unknown element "reset-dirs"


In [18]:
import traci

# After traci.start(sumoCmd)...

# Get current simulation time
current_time = traci.simulation.getTime()  # In seconds
print(f"Simulation time: {current_time}s")

# Get time step length
delta_t = traci.simulation.getDeltaT()  # In seconds
print(f"Step length: {delta_t}s")

# Get number of loaded/running/waiting vehicles
loaded = traci.simulation.getLoadedNumber()
departed = traci.simulation.getDepartedNumber()
arrived = traci.simulation.getArrivedNumber()
running = traci.vehicle.getIDCount()

print(f"Loaded: {loaded}, Departed: {departed}, Arrived: {arrived}")
print(f"Currently running: {running}")

# Get minimum expected number of vehicles
# (includes running, loaded, and inserted vehicles)
min_expected = traci.simulation.getMinExpectedNumber()
print(f"Minimum expected vehicles: {min_expected}")

# Run until all vehicles have left the simulation
# while traci.simulation.getMinExpectedNumber() > 0:
    # traci.simulationStep()
    # Your logic here...

# Get all current vehicle IDs
vehicle_ids = traci.vehicle.getIDList()
print(f"Current vehicles: {vehicle_ids}")

# Get all edge IDs
edge_ids = traci.edge.getIDList()
print(f"Number of edges: {len(edge_ids)}")

# Get all traffic light IDs  
tls_ids = traci.trafficlight.getIDList()
print(f"Traffic lights: {tls_ids}")

# Get network boundary
boundary = traci.simulation.getNetBoundary()
# Returns ((minX, minY), (maxX, maxY))
print(f"Network boundary: {boundary}")

Simulation time: 1000.0s
Step length: 1.0s
Loaded: 23, Departed: 0, Arrived: 0
Currently running: 731
Minimum expected vehicles: 22231
Current vehicles: ('flow_1_to_18.0', 'flow_1_to_18.1', 'flow_1_to_18.10', 'flow_1_to_18.100', 'flow_1_to_18.101', 'flow_1_to_18.102', 'flow_1_to_18.103', 'flow_1_to_18.104', 'flow_1_to_18.105', 'flow_1_to_18.106', 'flow_1_to_18.107', 'flow_1_to_18.108', 'flow_1_to_18.109', 'flow_1_to_18.11', 'flow_1_to_18.110', 'flow_1_to_18.111', 'flow_1_to_18.112', 'flow_1_to_18.113', 'flow_1_to_18.114', 'flow_1_to_18.115', 'flow_1_to_18.116', 'flow_1_to_18.117', 'flow_1_to_18.118', 'flow_1_to_18.119', 'flow_1_to_18.12', 'flow_1_to_18.120', 'flow_1_to_18.121', 'flow_1_to_18.122', 'flow_1_to_18.123', 'flow_1_to_18.124', 'flow_1_to_18.125', 'flow_1_to_18.126', 'flow_1_to_18.127', 'flow_1_to_18.128', 'flow_1_to_18.129', 'flow_1_to_18.13', 'flow_1_to_18.130', 'flow_1_to_18.131', 'flow_1_to_18.132', 'flow_1_to_18.133', 'flow_1_to_18.134', 'flow_1_to_18.135', 'flow_1_to_18.

In [19]:
traci.close()

---

## 16. TraCI - Adding Vehicles Dynamically <a name="traci-add-vehicles"></a>

In [ ]:
import traci

# Method 1: Add a trip (start and destination edges)
# Vehicle will compute route automatically

# First, define the route (as a trip)
traci.route.add("trip1", ["startEdge", "endEdge"])

# Then add the vehicle
traci.vehicle.add("newVeh1", "trip1", typeID="DEFAULT_VEHTYPE")

# The vehicle will compute full route using current travel times

# Method 2: Add vehicle with complete route
# Define route with all intermediate edges
edge_list = ["edge1", "edge2", "edge3", "edge4"]
traci.route.add("route1", edge_list)
traci.vehicle.add("newVeh2", "route1", typeID="DEFAULT_VEHTYPE")

# Method 3: Add vehicle with additional parameters
traci.vehicle.add(
    vehID="newVeh3",
    routeID="route1",
    typeID="passenger",
    depart="now",          # Departure time
    departLane="best",     # Lane selection
    departPos="base",      # Position on first edge
    departSpeed="0",       # Initial speed
    arrivalLane="current", # Arrival lane
    arrivalPos="max",      # Arrival position
    arrivalSpeed="current" # Arrival speed
)

# Method 4: Add with specific departure position
traci.vehicle.add("newVeh4", "route1")
# Immediately set position after adding
traci.vehicle.moveTo("newVeh4", "edge1_0", 50.0)  # Lane edge1_0, position 50m

# Example: Add vehicles at regular intervals
step = 0
while traci.simulation.getMinExpectedNumber() > 0:
    traci.simulationStep()
    
    # Add a new vehicle every 10 simulation steps
    if step % 10 == 0:
        veh_id = f"dynamic_veh_{step}"
        try:
            traci.vehicle.add(veh_id, "trip1")
            print(f"Added vehicle {veh_id} at time {traci.simulation.getTime()}")
        except traci.exceptions.TraCIException as e:
            print(f"Could not add vehicle: {e}")
    
    step += 1


---

## 17. TraCI - Subscriptions <a name="traci-subscriptions"></a>

Subscriptions allow you to efficiently retrieve data for multiple vehicles/objects without making individual queries each step.

### 17.1 Basic Subscriptions

In [ ]:
import traci
import traci.constants as tc

# Start simulation
traci.start(["sumo", "-c", "my.sumocfg"])

# Subscribe to vehicle variables
veh_id = "vehicle_0"
traci.vehicle.subscribe(veh_id, [tc.VAR_ROAD_ID, tc.VAR_LANEPOSITION])

# First call returns immediately (empty or None before v1.18.0)
results = traci.vehicle.getSubscriptionResults(veh_id)
print(f"Initial results: {results}")

# Run simulation and get updated values
for step in range(3):
    print(f"\\n--- Step {step} ---")
    traci.simulationStep()
    
    # Get subscription results (automatically updated after each step)
    results = traci.vehicle.getSubscriptionResults(veh_id)
    
    if results:
        road_id = results[tc.VAR_ROAD_ID]
        lane_pos = results[tc.VAR_LANEPOSITION]
        print(f"Vehicle {veh_id}:")
        print(f"  Road: {road_id}")
        print(f"  Position: {lane_pos:.2f}m")

# Unsubscribe (optional, automatic on close)
traci.vehicle.unsubscribe(veh_id)

traci.close()


---

## 18. TraCI - Context Subscriptions <a name="traci-context"></a>

Context subscriptions retrieve variables for all objects within a certain range of a reference object.

In [ ]:
import traci
import traci.constants as tc

traci.start(["sumo", "-c", "my.sumocfg"])

# Example 1: Subscribe to vehicles near a junction
junction_id = "junction_1"
radius = 50.0  # 50 meters

# Get all vehicles within 50m of the junction, retrieve their speed and waiting time
traci.junction.subscribeContext(
    junction_id,
    tc.CMD_GET_VEHICLE_VARIABLE,  # Type of objects to retrieve
    radius,                         # Search radius (meters)
    [tc.VAR_SPEED, tc.VAR_WAITING_TIME]  # Variables to retrieve
)

# Get initial results
results = traci.junction.getContextSubscriptionResults(junction_id)
print(f"Initial context results: {results}")

# Run simulation
for step in range(3):
    print(f"\\n--- Step {step} ---")
    traci.simulationStep()
    
    # Get results: dict mapping vehicle IDs to their variable values
    context_results = traci.junction.getContextSubscriptionResults(junction_id)
    
    if context_results:
        print(f"Vehicles near {junction_id}:")
        for veh_id, data in context_results.items():
            speed = data[tc.VAR_SPEED]
            waiting = data[tc.VAR_WAITING_TIME]
            print(f"  {veh_id}: speed={speed:.2f}m/s, waiting={waiting:.2f}s")
    else:
        print(f"No vehicles near {junction_id}")

# Example 2: Get all vehicles in the entire network
# Use a large radius or subscribe at a central point
junction_id = traci.junction.getIDList()[0]  # Any junction
large_radius = 1000000.0  # Very large radius

traci.junction.subscribeContext(
    junction_id,
    tc.CMD_GET_VEHICLE_VARIABLE,
    large_radius,
    [tc.VAR_SPEED, tc.VAR_ALLOWED_SPEED]
)

# This effectively gets all vehicles in the network
step_length = traci.simulation.getDeltaT()

while traci.simulation.getMinExpectedNumber() > 0:
    traci.simulationStep()
    
    sc_results = traci.junction.getContextSubscriptionResults(junction_id)
    
    if sc_results:
        # Compute network-wide statistics
        speeds = [d[tc.VAR_SPEED] for d in sc_results.values()]
        allowed_speeds = [d[tc.VAR_ALLOWED_SPEED] for d in sc_results.values()]
        rel_speeds = [v/a for v, a in zip(speeds, allowed_speeds)]
        
        running = len(rel_speeds)
        halting = sum(1 for d in sc_results.values() if d[tc.VAR_SPEED] < 0.1)
        mean_speed_relative = sum(rel_speeds) / running if running > 0 else 0
        time_loss = (1 - mean_speed_relative) * running * step_length
        
        print(f"Time: {traci.simulation.getTime():.1f}s, "
              f"TimeLoss: {time_loss:.2f}s, Halting: {halting}")

traci.close()


---

## 19. TraCI - Subscription Filters <a name="traci-filters"></a>

Filters refine vehicle-to-vehicle context subscriptions to specific lanes or maneuvers.

In [ ]:
import traci
import traci.constants as tc

traci.start(["sumo", "-c", "my.sumocfg"])

ego_id = "ego_vehicle"

# Example 1: Lane filter - vehicles on specific lanes relative to ego
# Subscribe to vehicles around ego (radius can be 0, overridden by filter)
traci.vehicle.subscribeContext(ego_id, tc.CMD_GET_VEHICLE_VARIABLE, 0.0, [tc.VAR_SPEED])

# Add lane filter: get vehicles on current lane and adjacent lanes
lanes = [0, 1, -1]  # 0=current, 1=right, -1=left
downstream_dist = 100.0  # meters ahead
upstream_dist = 50.0     # meters behind
no_opposite = True       # Don't include opposite direction

traci.vehicle.addSubscriptionFilterLanes(
    lanes,
    noOpposite=no_opposite,
    downstreamDist=downstream_dist,
    upstreamDist=upstream_dist
)

# Now only vehicles on specified lanes within distance are returned

# Example 2: Car-Following maneuver filter
# Get leader and follower on current lane
traci.vehicle.subscribeContext(ego_id, tc.CMD_GET_VEHICLE_VARIABLE, 0.0, 
                               [tc.VAR_SPEED, tc.VAR_LANEPOSITION])
traci.vehicle.addSubscriptionFilterCFManeuver()

# Returns at most 2 vehicles: leader and follower

# Example 3: Lane-Change maneuver filter
# Get leaders and followers on current and target lane
traci.vehicle.subscribeContext(ego_id, tc.CMD_GET_VEHICLE_VARIABLE, 0.0,
                               [tc.VAR_SPEED])
traci.vehicle.addSubscriptionFilterLCManeuver(
    direction=1,  # -1=left, 0=current, 1=right, 2=both
    noOpposite=True,
    downstreamDist=100.0,
    upstreamDist=50.0
)

# Returns relevant vehicles for lane change decision

# Example 4: Turn filter - vehicles on conflicting paths at junction
traci.vehicle.subscribeContext(ego_id, tc.CMD_GET_VEHICLE_VARIABLE, 0.0,
                               [tc.VAR_SPEED])
traci.vehicle.addSubscriptionFilterTurn(downstreamDist=100.0)

# Returns vehicles that will conflict at upcoming junctions

# Example 5: Vehicle type filter
traci.vehicle.subscribeContext(ego_id, tc.CMD_GET_VEHICLE_VARIABLE, 50.0,
                               [tc.VAR_SPEED])
traci.vehicle.addSubscriptionFilterVType(["truck", "bus"])

# Only returns trucks and buses, filters out other types

# Run simulation
while traci.simulation.getMinExpectedNumber() > 0:
    traci.simulationStep()
    
    results = traci.vehicle.getContextSubscriptionResults(ego_id)
    if results:
        print(f"Filtered vehicles: {list(results.keys())}")

traci.close()


---

## 20. TraCI - Vehicle Information <a name="traci-vehicle-info"></a>

### 20.1 Getting Vehicle Information

In [ ]:
import traci

# After starting simulation and with vehicles present...

veh_id = "vehicle_0"

# Position and movement
position = traci.vehicle.getPosition(veh_id)  # (x, y) coordinates
speed = traci.vehicle.getSpeed(veh_id)        # m/s
acceleration = traci.vehicle.getAcceleration(veh_id)  # m/s²
angle = traci.vehicle.getAngle(veh_id)        # degrees

print(f"Vehicle {veh_id}:")
print(f"  Position: {position}")
print(f"  Speed: {speed:.2f} m/s")
print(f"  Acceleration: {acceleration:.2f} m/s²")
print(f"  Angle: {angle:.2f}°")

# Road information
road_id = traci.vehicle.getRoadID(veh_id)     # Current edge
lane_id = traci.vehicle.getLaneID(veh_id)     # Current lane
lane_index = traci.vehicle.getLaneIndex(veh_id)  # Lane index
lane_pos = traci.vehicle.getLanePosition(veh_id)  # Position on lane (m)

print(f"  Road: {road_id}")
print(f"  Lane: {lane_id} (index {lane_index})")
print(f"  Lane position: {lane_pos:.2f}m")

# Route information
route = traci.vehicle.getRoute(veh_id)        # List of edge IDs
route_index = traci.vehicle.getRouteIndex(veh_id)  # Current edge index
route_id = traci.vehicle.getRouteID(veh_id)   # Route ID

print(f"  Route ID: {route_id}")
print(f"  Route: {route}")
print(f"  Current edge in route: {route_index}")

# Speed and limits
max_speed = traci.vehicle.getMaxSpeed(veh_id)  # Vehicle's max speed
allowed_speed = traci.vehicle.getAllowedSpeed(veh_id)  # Legal limit
speed_factor = traci.vehicle.getSpeedFactor(veh_id)  # Speed factor

print(f"  Max speed: {max_speed:.2f} m/s")
print(f"  Allowed speed: {allowed_speed:.2f} m/s")
print(f"  Speed factor: {speed_factor:.2f}")

# Following and gaps
leader = traci.vehicle.getLeader(veh_id, dist=100.0)
if leader:
    leader_id, distance = leader
    print(f"  Leader: {leader_id} at {distance:.2f}m")
else:
    print(f"  No leader within 100m")

# Waiting and stops
waiting_time = traci.vehicle.getWaitingTime(veh_id)  # Seconds
accumulated_wait = traci.vehicle.getAccumulatedWaitingTime(veh_id)

print(f"  Waiting time: {waiting_time:.2f}s")
print(f"  Accumulated waiting: {accumulated_wait:.2f}s")

# Vehicle type and class
veh_type = traci.vehicle.getTypeID(veh_id)
veh_class = traci.vehicle.getVehicleClass(veh_id)

print(f"  Type: {veh_type}")
print(f"  Class: {veh_class}")

# Emissions
co2 = traci.vehicle.getCO2Emission(veh_id)    # mg/s
fuel = traci.vehicle.getFuelConsumption(veh_id)  # ml/s

print(f"  CO2: {co2:.2f} mg/s")
print(f"  Fuel: {fuel:.2f} ml/s")


---

## 21. TraCI - Vehicle Control <a name="traci-vehicle-control"></a>

### 21.1 Controlling Vehicle Behavior

In [ ]:
import traci

veh_id = "vehicle_0"

# Speed control
traci.vehicle.setSpeed(veh_id, 15.0)  # Set speed to 15 m/s
traci.vehicle.slowDown(veh_id, 10.0, 5.0)  # Slow to 10 m/s over 5 seconds
traci.vehicle.setSpeed(veh_id, -1)  # Return to normal driving (automatic)

# Speed factor (affects desired speed)
traci.vehicle.setSpeedFactor(veh_id, 1.2)  # Drive 20% faster than limit
traci.vehicle.setMaxSpeed(veh_id, 30.0)    # Set absolute max speed

# Lane changes
traci.vehicle.changeLane(veh_id, lane_index=1, duration=3.0)
# Change to lane 1 over 3 seconds

# Sublane model (lateral position)
traci.vehicle.changeSublane(veh_id, 1.5)  # Shift 1.5m laterally

# Route changes
new_route = ["edge1", "edge2", "edge3", "edge4"]
traci.vehicle.setRoute(veh_id, new_route)

# Add to existing route
traci.vehicle.setRouteID(veh_id, "route_id")

# Change to new destination (automatic rerouting)
traci.vehicle.changeTarget(veh_id, "new_destination_edge")

# Reroute based on current travel times
traci.vehicle.rerouteTraveltime(veh_id)

# Stops (bus stops, parking, etc.)
traci.vehicle.setStop(
    veh_id,
    edgeID="edge1",
    pos=100.0,           # Position on edge (m)
    laneIndex=0,         # Lane index
    duration=30.0,       # Stop duration (s)
    flags=0,             # Stop flags
    startPos=90.0,       # Start of stopping range
    until=-1             # Time until which to stop (-1 = duration)
)

# Color (for visualization)
traci.vehicle.setColor(veh_id, (255, 0, 0, 255))  # Red (RGBA)

# Vehicle type change
traci.vehicle.setType(veh_id, "new_type_id")

# Teleport (move instantly)
traci.vehicle.moveTo(veh_id, "edge2_0", 50.0)  # To lane edge2_0 at position 50m

# Remove from simulation
traci.vehicle.remove(veh_id, reason=traci.constants.REMOVE_VAPORIZED)

# Example: Adaptive Cruise Control
ego = "ego_vehicle"
leader_info = traci.vehicle.getLeader(ego, 100.0)

if leader_info:
    leader_id, distance = leader_info
    leader_speed = traci.vehicle.getSpeed(leader_id)
    
    # Simple ACC logic
    safe_distance = 20.0  # meters
    
    if distance < safe_distance:
        # Too close, slow down
        target_speed = max(0, leader_speed - 2.0)
        traci.vehicle.slowDown(ego, target_speed, 1.0)
    elif distance > safe_distance + 10:
        # Far enough, speed up
        target_speed = min(leader_speed + 2.0, 
                          traci.vehicle.getAllowedSpeed(ego))
        traci.vehicle.slowDown(ego, target_speed, 2.0)



---

## 22. Advanced Topics - Parallel Simulations <a name="parallel"></a>

Running multiple simulations simultaneously:

In [ ]:
import traci

# Method 1: Using labels and switching
traci.start(["sumo", "-c", "sim1.sumocfg"], label="sim1")
traci.start(["sumo", "-c", "sim2.sumocfg"], label="sim2")

# Switch between simulations
traci.switch("sim1")
traci.simulationStep()  # Step sim1

traci.switch("sim2")
traci.simulationStep()  # Step sim2

# Close both
traci.switch("sim1")
traci.close()
traci.switch("sim2")
traci.close()

# Method 2: Using connection objects (more object-oriented)
traci.start(["sumo", "-c", "sim1.sumocfg"], label="sim1")
traci.start(["sumo", "-c", "sim2.sumocfg"], label="sim2")

conn1 = traci.getConnection("sim1")
conn2 = traci.getConnection("sim2")

# Use connection objects
conn1.simulationStep()  # Step sim1
conn2.simulationStep()  # Step sim2

# Access vehicle info through connections
vehicles1 = conn1.vehicle.getIDList()
vehicles2 = conn2.vehicle.getIDList()

print(f"Sim1 vehicles: {len(vehicles1)}")
print(f"Sim2 vehicles: {len(vehicles2)}")

# Close
conn1.close()
conn2.close()


---

## 23. Advanced Topics - Error Handling and Debugging <a name="error-handling"></a>

In [ ]:
import traci
import sys

# 1. Handle connection errors
try:
    traci.start(["sumo", "-c", "config.sumocfg"])
except traci.exceptions.FatalTraCIError as e:
    print(f"Could not connect to SUMO: {e}")
    print("Check that:")
    print("  1. SUMO is installed correctly")
    print("  2. Configuration file exists")
    print("  3. Network files are valid")
    sys.exit(1)

# 2. Handle TraCI exceptions during simulation
veh_id = "vehicle_0"

try:
    position = traci.vehicle.getPosition(veh_id)
except traci.exceptions.TraCIException as e:
    print(f"TraCI error: {e}")
    # Vehicle might not exist yet or has already left
    pass

# 3. Safe vehicle control
def safe_vehicle_control(veh_id, speed):
    try:
        traci.vehicle.setSpeed(veh_id, speed)
        return True
    except traci.exceptions.TraCIException:
        # Vehicle doesn't exist or can't be controlled
        return False

# 4. Logging for debugging
# Add logging to SUMO command
sumo_cmd = ["sumo", "-c", "config.sumocfg", 
            "--log", "sumo_log.txt",
            "--message-log", "messages.txt",
            "--error-log", "errors.txt"]

traci.start(sumo_cmd)

# 5. Trace file for reproducing issues
# Records all TraCI commands
traci.start(["sumo", "-c", "config.sumocfg"],
            traceFile="traci_trace.txt",
            traceGetters=True)  # Include getter calls in trace

# 6. Check SUMO version compatibility
print(f"TraCI API version: {traci.getVersion()}")

# 7. Graceful shutdown
def run_simulation():
    try:
        traci.start(["sumo", "-c", "config.sumocfg"])
        
        while traci.simulation.getMinExpectedNumber() > 0:
            traci.simulationStep()
            # Your simulation logic...
            
    except KeyboardInterrupt:
        print("Simulation interrupted by user")
    except Exception as e:
        print(f"Simulation error: {e}")
    finally:
        # Always close TraCI connection
        if traci.isLoaded():
            traci.close()
        print("Simulation ended")

# 8. Debugging vehicle behavior
def debug_vehicle(veh_id):
    '''Print detailed vehicle state for debugging'''
    try:
        print(f"\\nVehicle {veh_id} state:")
        print(f"  Position: {traci.vehicle.getPosition(veh_id)}")
        print(f"  Speed: {traci.vehicle.getSpeed(veh_id):.2f} m/s")
        print(f"  Road: {traci.vehicle.getRoadID(veh_id)}")
        print(f"  Lane: {traci.vehicle.getLaneID(veh_id)}")
        print(f"  Route: {traci.vehicle.getRoute(veh_id)}")
        print(f"  RouteIndex: {traci.vehicle.getRouteIndex(veh_id)}")
    except traci.exceptions.TraCIException as e:
        print(f"  Error getting vehicle info: {e}")


---

## 24. Advanced Topics - Best Practices and Performance Tips <a name="best-practices"></a>

In [ ]:
# 1. EFFICIENT DATA RETRIEVAL: Use subscriptions for repeated queries
# BAD: Query individual values each step
for step in range(1000):
    traci.simulationStep()
    for veh_id in traci.vehicle.getIDList():
        speed = traci.vehicle.getSpeed(veh_id)  # Multiple network calls
        pos = traci.vehicle.getPosition(veh_id)

# GOOD: Use context subscriptions
import traci.constants as tc
junction_id = traci.junction.getIDList()[0]
traci.junction.subscribeContext(junction_id, tc.CMD_GET_VEHICLE_VARIABLE,
                                1000000, [tc.VAR_SPEED, tc.VAR_POSITION])
for step in range(1000):
    traci.simulationStep()
    results = traci.junction.getContextSubscriptionResults(junction_id)
    # All vehicle data retrieved in one call

# 2. BATCH OPERATIONS: Group similar operations
# BAD: Add vehicles one at a time with multiple operations
for i in range(100):
    traci.route.add(f"route_{i}", ["e1", "e2"])
    traci.vehicle.add(f"veh_{i}", f"route_{i}")
    traci.vehicle.setSpeed(f"veh_{i}", 20.0)

# GOOD: Minimize TraCI calls, prepare data first
routes = {f"route_{i}": ["e1", "e2"] for i in range(100)}
for route_id, edges in routes.items():
    traci.route.add(route_id, edges)
vehicles = [(f"veh_{i}", f"route_{i}", 20.0) for i in range(100)]
for veh_id, route_id, speed in vehicles:
    traci.vehicle.add(veh_id, route_id)
    if speed != -1:
        traci.vehicle.setSpeed(veh_id, speed)

# 3. MEMORY MANAGEMENT: Don't store unnecessary data
# BAD: Store all historical data
vehicle_history = {}
while traci.simulation.getMinExpectedNumber() > 0:
    traci.simulationStep()
    for veh_id in traci.vehicle.getIDList():
        if veh_id not in vehicle_history:
            vehicle_history[veh_id] = []
        vehicle_history[veh_id].append(traci.vehicle.getPosition(veh_id))

# GOOD: Process and aggregate, keep only what's needed
vehicle_distances = {}  # Only store total distance
while traci.simulation.getMinExpectedNumber() > 0:
    traci.simulationStep()
    for veh_id in traci.vehicle.getIDList():
        distance = traci.vehicle.getDistance(veh_id)
        vehicle_distances[veh_id] = distance

# 4. ERROR HANDLING: Always use try-except for optional operations
# GOOD: Defensive programming
for veh_id in potential_vehicle_ids:
    try:
        speed = traci.vehicle.getSpeed(veh_id)
        # Process speed...
    except traci.exceptions.TraCIException:
        # Vehicle doesn't exist, skip
        continue

# 5. RESOURCE CLEANUP: Always close connections
try:
    traci.start(sumoCmd)
    # Simulation code...
finally:
    if traci.isLoaded():
        traci.close()

# 6. CONFIGURATION: Use configuration objects for complex scenarios
from dataclasses import dataclass

@dataclass
class SimConfig:
    network_file: str
    route_file: str
    step_length: float = 0.1
    seed: int = 42
    gui: bool = False
    
    def to_sumo_cmd(self):
        binary = "sumo-gui" if self.gui else "sumo"
        cmd = [binary,
               "--net-file", self.network_file,
               "--route-files", self.route_file,
               "--step-length", str(self.step_length),
               "--seed", str(self.seed)]
        return cmd

config = SimConfig(network_file="net.net.xml", route_file="routes.rou.xml")
traci.start(config.to_sumo_cmd())

# 7. MODULARITY: Separate concerns
class VehicleController:
    def __init__(self, veh_id):
        self.veh_id = veh_id
    
    def get_state(self):
        return {
            'speed': traci.vehicle.getSpeed(self.veh_id),
            'position': traci.vehicle.getPosition(self.veh_id),
            'lane': traci.vehicle.getLaneID(self.veh_id)
        }
    
    def set_speed(self, speed):
        traci.vehicle.setSpeed(self.veh_id, speed)

# 8. DOCUMENTATION: Comment complex logic
def compute_time_loss(results, step_length):
    '''
    Compute total time loss for vehicles in context subscription results.
    
    Time loss = time spent driving below maximum speed
    Formula: (1 - actual_speed/max_speed) * num_vehicles * step_length
    
    Args:
        results: Context subscription results dict
        step_length: Simulation step length in seconds
    Returns:
        Time loss in seconds
    '''
    if not results:
        return 0.0
    
    rel_speeds = [d[tc.VAR_SPEED] / d[tc.VAR_ALLOWED_SPEED] 
                  for d in results.values()]
    mean_rel_speed = sum(rel_speeds) / len(rel_speeds)
    time_loss = (1 - mean_rel_speed) * len(results) * step_length
    return time_loss


---

## 25. Complete Example - Adaptive Traffic Light Control System <a name="complete-example"></a>

In [ ]:
import os
import sys
import traci
import traci.constants as tc

# Setup
if 'SUMO_HOME' in os.environ:
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))

class AdaptiveTrafficLight:
    '''Adaptive traffic light controller based on vehicle detection'''
    
    def __init__(self, tls_id, detector_ids, min_green=10, max_green=60):
        self.tls_id = tls_id
        self.detector_ids = detector_ids
        self.min_green = min_green  # Minimum green time (seconds)
        self.max_green = max_green  # Maximum green time (seconds)
        self.current_phase = 0
        self.phase_start_time = 0
        
    def update(self, current_time):
        '''Update traffic light based on detector readings'''
        
        # Get current phase duration
        phase_duration = current_time - self.phase_start_time
        
        # Don't change if minimum green time not reached
        if phase_duration < self.min_green:
            return
        
        # Check if we should extend or switch phase
        vehicles_detected = sum(
            traci.inductionloop.getLastStepVehicleNumber(det)
            for det in self.detector_ids
        )
        
        # Switch if no vehicles detected or max green reached
        if vehicles_detected == 0 or phase_duration >= self.max_green:
            self.switch_phase()
            self.phase_start_time = current_time
    
    def switch_phase(self):
        '''Switch to next traffic light phase'''
        num_phases = len(traci.trafficlight.getAllProgramLogics(self.tls_id)[0].phases)
        self.current_phase = (self.current_phase + 1) % num_phases
        traci.trafficlight.setPhase(self.tls_id, self.current_phase)
        print(f"Switched TLS {self.tls_id} to phase {self.current_phase}")


def run_simulation():
    '''Run simulation with adaptive traffic light control'''
    
    # Configuration
    sumo_binary = "sumo"  # Use "sumo-gui" for visualization
    sumo_config = "simulation.sumocfg"
    
    sumo_cmd = [
        sumo_binary,
        "-c", sumo_config,
        "--step-length", "1.0",
        "--no-warnings", "true"
    ]
    
    try:
        # Start simulation
        traci.start(sumo_cmd)
        print("Simulation started")
        
        # Initialize traffic light controllers
        tls_controllers = {}
        for tls_id in traci.trafficlight.getIDList():
            # In real scenario, you'd specify which detectors belong to each TLS
            detector_ids = ["detector_1", "detector_2"]  # Example
            tls_controllers[tls_id] = AdaptiveTrafficLight(tls_id, detector_ids)
        
        # Setup subscriptions for efficiency
        junction_id = traci.junction.getIDList()[0]
        traci.junction.subscribeContext(
            junction_id,
            tc.CMD_GET_VEHICLE_VARIABLE,
            1000000,  # Large radius to get all vehicles
            [tc.VAR_SPEED, tc.VAR_WAITING_TIME, tc.VAR_ALLOWED_SPEED]
        )
        
        step_length = traci.simulation.getDeltaT()
        
        # Statistics
        total_waiting_time = 0
        total_time_loss = 0
        step_count = 0
        
        # Main simulation loop
        print("Running simulation...")
        while traci.simulation.getMinExpectedNumber() > 0:
            traci.simulationStep()
            current_time = traci.simulation.getTime()
            
            # Update traffic lights
            for controller in tls_controllers.values():
                controller.update(current_time)
            
            # Collect statistics
            results = traci.junction.getContextSubscriptionResults(junction_id)
            if results:
                # Total waiting time
                waiting = sum(d[tc.VAR_WAITING_TIME] for d in results.values())
                total_waiting_time += waiting
                
                # Time loss calculation
                rel_speeds = [
                    d[tc.VAR_SPEED] / d[tc.VAR_ALLOWED_SPEED]
                    for d in results.values()
                    if d[tc.VAR_ALLOWED_SPEED] > 0
                ]
                if rel_speeds:
                    mean_rel_speed = sum(rel_speeds) / len(rel_speeds)
                    time_loss = (1 - mean_rel_speed) * len(results) * step_length
                    total_time_loss += time_loss
            
            step_count += 1
            
            # Progress update every 100 steps
            if step_count % 100 == 0:
                print(f"Step {step_count}: Time={current_time:.1f}s, "
                      f"Vehicles={len(results) if results else 0}")
        
        # Print final statistics
        print("\\nSimulation completed!")
        print(f"Total steps: {step_count}")
        print(f"Total waiting time: {total_waiting_time:.2f}s")
        print(f"Total time loss: {total_time_loss:.2f}s")
        print(f"Average waiting per step: {total_waiting_time/step_count:.2f}s")
        
    except traci.exceptions.FatalTraCIError as e:
        print(f"Fatal TraCI error: {e}")
        print("Check configuration and network files")
        
    except KeyboardInterrupt:
        print("\\nSimulation interrupted by user")
        
    except Exception as e:
        print(f"Unexpected error: {e}")
        import traceback
        traceback.print_exc()
        
    finally:
        # Always close TraCI
        if traci.isLoaded():
            traci.close()
            print("TraCI connection closed")


if __name__ == "__main__":
    run_simulation()


---

## Summary and Quick Reference

## Conclusion

This notebook provides a comprehensive guide to **SUMO** and **sumolib** for traffic simulation with Python. You've learned:

1. **sumolib basics**: Network loading, element access, XML parsing, coordinates
2. **Network creation**: Programmatic generation of networks and routes
3. **TraCI fundamentals**: Connecting, controlling simulations, retrieving data
4. **Advanced features**: Subscriptions, filters, parallel simulations
5. **Best practices**: Performance optimization, error handling, code structure

### Next Steps

1. **Practice**: Set up your own SUMO scenarios and experiment
2. **Explore**: Try different control algorithms and analysis techniques
3. **Extend**: Build upon the examples for your specific use cases
4. **Optimize**: Profile your code and apply performance improvements

### Key Takeaways

- Use **sumolib** for static network analysis and file processing
- Use **TraCI** for runtime control and interactive simulations
- **Subscriptions** are essential for efficient data retrieval
- Always handle errors and clean up resources properly
- Start with **sumo-gui** for debugging, switch to **sumo** for production
- Consider **libsumo** for performance-critical applications

### Additional Learning Resources

- **SUMO Tutorials**: https://sumo.dlr.de/docs/Tutorials.html
- **User Conference Videos**: https://sumo.dlr.de/docs/Conference.html
- **GitHub Examples**: https://github.com/eclipse-sumo/sumo/tree/main/tests
- **Mailing List**: sumo-user@eclipse.org
- **Stack Overflow**: Tag `sumo`

---

**Created**: December 2025  
**Last Updated**: December 2025  
**Version**: 1.0

Happy Simulating! 🚗🚕🚙